In [1]:
import os, shutil, csv, hashlib
import os
import glob
#import numpy as np
import pandas as pd
from IPython.display import display

import re

#from Bio.Blast import NCBIWWW, NCBIXML


In [2]:
!hostname 

n1.hpc.at.sfsu.edu


In [9]:
!free -h  

              total        used        free      shared  buff/cache   available
Mem:          188Gi        80Gi        59Gi       222Mi        48Gi       106Gi
Swap:         2.0Gi       2.0Gi       0.0Ki


In [5]:
INPUT_DIR  = "/Users/921623492/Ecoli_Project/Data/merged_alignment"
OUTPUT_DIR = "/Users/921623492/Ecoli_Project/Data/MAF_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [6]:
file_paths = glob.glob(INPUT_DIR+"/*_pangenome_alignment.npy")
if len(file_paths) == 0:
    raise FileNotFoundError("No .npy files found in the 'alignment' directory!")

In [7]:
len(file_paths)

600

In [8]:
# Reduced pangenome fasta file 
fasta_path = "/Users/921623492/Ecoli_Project/Data/reduced_pangenome_blast.fa"

In [9]:
def extract_gene_names_from_fasta(fasta_path):
    with open(fasta_path, "r") as f:
        gene_names = [line[1:].strip() for line in f if line.startswith(">")]
    return gene_names

In [10]:
gene_names = extract_gene_names_from_fasta(fasta_path)

In [1]:
#gene_names

In [10]:
gene_names = gene_names[:15629]

In [9]:
combined_matrix = []
sample_ids = []


for path in file_paths:
    arr = np.load(path, allow_pickle=True)

    combined_matrix.append(arr)

    # Extract sample ID from filename
    sample_id = os.path.basename(path).split(".")[0]
    sample_ids.append(sample_id)

# Convert to DataFrame (genes as columns, samples as rows)
combined_df = pd.DataFrame(np.array(combined_matrix), 
                           index=sample_ids,
                           columns=gene_names)
combined_df.index.name = "Sample_ID"

In [10]:
combined_df.head()

,FAHFDEJI_02221,GNOIHJGF_04610,NHLJOMBL_05016,KCNLHOEA_03403,HHCOCAHC_02102,KMKPFFJF_01017,MAOGMHBA_04903,CKMBOFAF_03765,FAHFDEJI_00308,ECKICHIN_03110,...,NHLJOMBL_00571,GHOPNDEC_03154,AEGEGBCH_03641,DMMNDODG_04815,EDEFNFDI_05015,EDEFNFDI_05016,BPDJECED_04485,DMLEFFEM_04762,EDHNABJO_03523,LBOLKHKO_00606
Sample_ID,,,,,,,,,,,,,,,,,,,,,
mills_cip_200__GCA_025783615,"[a, t, g, t, c, t, a, t, a, c, a, g, a, a, c, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[a, t, g, a, g, t, a, a, g, a, t, t, a, t, c, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[a, t, g, c, g, g, a, a, g, a, g, t, a, t, g, ...","[a, t, g, c, g, g, a, a, g, a, g, t, a, t, g, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[a, t, g, t, c, c, c, a, g, a, t, a, g, a, a, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
kallonen_cip_200__ERR434265_pangenome_alignment,"[a, t, g, t, c, t, a, t, a, c, a, g, a, a, c, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[a, t, g, a, g, t, a, a, g, a, t, t, a, t, c, ...","[a, t, g, c, c, a, c, t, a, a, t, t, a, t, c, ...",...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[a, t, g, c, g, g, a, a, g, a, g, t, a, t, g, ...","[a, t, g, c, g, g, a, a, g, a, g, t, a, t, g, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[a, t, g, t, c, c, c, a, g, a, t, a, g, a, a, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
kallonen_cip_200__ERR439575_pangenome_alignment,"[a, t, g, t, c, t, a, t, a, c, a, g, a, a, c, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[a, t, g, a, g, t, a, a, g, a, t, t, a, t, c, ...","[a, t, g, c, c, a, c, t, a, a, t, t, a, t, c, ...",...,"[-, -, -, -, -, -, -, -, -, -, -, -, -, -, -, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[a, t, g, c, g, g, a, a, g, a, g, t, a, t, g, ...","[a, t, g, c, g, g, a, a, g, a, g, t, a, t, g, ...","[-, -, -, -, -, -, c, t, g, t, a, c, a, a, t, ...","[a, t, g, t, c, c, c, a, g, a, t, a, g, a, a, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
gladstone-cip-200__ERR4036619_pangenome_alignment,"[a, t, g, t, c, t, a, t, a, c, a, g, a, a, c, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[a, t, g, a, a, c, g, t, t, a, t, a, a, a, a

### Step 1:  Extract your DNA sequence from the DataFrame


In [ ]:
gene_id = "FAHFDEJI_02221"
gene_id

'MAOGMHBA_04903'

In [17]:
char_Array = combined_df[gene_id].dropna().iloc[0]

In [18]:
char_Array

array([nan, nan, nan, ..., nan, nan, nan])

In [19]:
len(char_Array)

2262

In [ ]:
sequence = "".join(char_Array)
sequence

TypeError: sequence item 0: expected str instance, numpy.float64 found

: 

In [ ]:
# build a Fasta-formatted string 
fasta_query = f">{gene_id}\n{sequence}\n"

In [ ]:
fasta_query

### Step 2: Submit your FASTA to NCBI’s remote BLAST


##### Experiment 1

In [ ]:
from Bio.Blast import NCBIWWW, NCBIXML 

print("Sending to NCBI Blast this can take 10-30 s")
result_handle = NCBIWWW.qblast(
    program="blastn",
    database="nt",
    sequence=fasta_query,
    format_type="XML",
    hitlist_size=5
)


In [ ]:
# 1. Parse the XML response into Blast record(s)
blast_records = NCBIXML.parse(result_handle)
blast_record  = next(blast_records) 

In [ ]:
# 2. Grab the very first alignment (best‐scoring hit)
best_alignment = blast_record.alignments[0]
best_hsp       = best_alignment.hsps[0]  # high‐scoring pair

In [ ]:
# 3. Extract the pieces you care about
hit_id    = best_alignment.hit_id        # e.g. "gi|123456789|ref|WP_012345678.1|"
hit_def   = best_alignment.hit_def       # full title line
evalue    = best_hsp.expect              # e‐value
bitscore  = best_hsp.bits                # bitscore
pid       = best_hsp.identities          # number of identical residues
align_len = best_hsp.align_length        # alignment length

In [ ]:

# 4. Print your summary
print(f"Top hit: {hit_id}")
print(f" > {hit_def}")
print(f" e‐value: {evalue}, bitscore: {bitscore}")
print(f" identity: {pid}/{align_len} ({pid/align_len*100:.1f}%)")

In [ ]:

# 5. Clean up
result_handle.close()

##### Experiment 2 

In [ ]:

# blastn against RefSeq RNA for E. coli
handle_2 = NCBIWWW.qblast(
    program     = "blastr",
    database    = "rn",
    sequence    = fasta_query,
    format_type = "XML",
    hitlist_size= 1,
    megablast   = True
)

In [ ]:

rec = next(NCBIXML.parse(handle_2))
handle_2.close()

if rec.alignments:
    defln = rec.alignments[0].hit_def
    # e.g. "NM_001126112.1 Escherichia coli gyrA gene, mRNA"
    gene_name = defln.split()[2]      # “gyrA”
    print("Gene symbol:", gene_name)
else:
    print("No transcript hit found.")

#### Experiment 3:  Grab it from a protein‐level search + “GN=” tag

In [ ]:
handle = NCBIWWW.qblast(
    program     = "blastn",
    database    = "nt",                  # all nucleotide sequences
    sequence    = fasta_query,
    entrez_query= "txid562[Organism]",   # still restrict to E. coli
    format_type = "XML",
    hitlist_size= 1,
    megablast   = True
)


In [ ]:
rec = next(NCBIXML.parse(handle))
handle.close()

if rec.alignments:
    defln = rec.alignments[0].hit_def
    acc = defln.split()[0]              # first token is the accession
    if acc.startswith(("NM_","NR_")):
        # a transcript
        gene_symbol = defln.split()[2]  # usually 3rd token is the gene name
        print("Gene symbol:", gene_symbol)
    else:
        print("Best hit wasn’t a transcript:", defln)
else:
    print("No nt hits found.")

#### Experiment 4: 

In [ ]:

print("🔍 Sending to NCBI BLAST… this can take ~10–30 s")
result_handle = NCBIWWW.qblast(
    program="blastn",      # nucleotide-vs-nucleotide search
    database="nt",         # NCBI’s non-redundant nucleotide database
    sequence=fasta_query,
    format_type="XML",     # so we can parse with NCBIXML
    hitlist_size=5         # return up to 5 top hits
)

In [ ]:
# 3.1 parse the XML into a Blast.Record
blast_record = next(NCBIXML.parse(result_handle))

In [ ]:
# 3.2 take the very first alignment (best hit)
best_alignment = blast_record.alignments[0]
hsp = best_alignment.hsps[0]   # the top high-scoring pair

In [ ]:
# 3.3 extract identifiers & stats
hit_id       = best_alignment.hit_id        # e.g. "gi|123456789|ref|NM_001..."
hit_def      = best_alignment.hit_def       # full title/description
percent_id   = hsp.identities / hsp.align_length * 100
evalue       = hsp.expect
bitscore     = hsp.bits

print("Top hit:")
print(" • hit_id:   ", hit_id)
print(" • defline:  ", hit_def)
print(f" • identity: {percent_id:.1f}%  length: {hsp.align_length}  e={evalue}")

In [ ]:
hit_id = "gi|2990726386|dbj|AP040229.1|"
acc    = hit_id.split("|")[-2]      # "AP040229.1"
start  = 1                         # replace with sbjct_start-1
end    = 579                       # replace with sbjct_end

In [ ]:
from Bio import Entrez, SeqIO
Entrez.email = "you@domain.com"

Entrez.email = "aabdillahid@gmail.com"
gb = Entrez.efetch(db="nuccore",
                  id=acc,
                  rettype="gb",
                  retmode="text")
record = SeqIO.read(gb, "gb")
gb.close()

# Find the CDS feature covering your alignment:
for feat in record.features:
    if feat.type == "CDS" and feat.location.start <= start < feat.location.end:
        gene_symbol = feat.qualifiers.get("gene", [""])[0]
        print("Gene symbol:", gene_symbol)
        break
else:
    print("No CDS covers that region.")

In [6]:
data = "/Users/921623492/Ecoli_Project/Experiment/results/combined_pangenome_matrix.csv"


df = pd.read_csv(data)

In [7]:
df.shape

(600, 15630)

In [8]:
df.shape

(600, 15630)

In [9]:
df

,Sample_ID,FAHFDEJI_02221,GNOIHJGF_04610,NHLJOMBL_05016,KCNLHOEA_03403,HHCOCAHC_02102,KMKPFFJF_01017,MAOGMHBA_04903,CKMBOFAF_03765,FAHFDEJI_00308,...,NHLJOMBL_00571,GHOPNDEC_03154,AEGEGBCH_03641,DMMNDODG_04815,EDEFNFDI_05015,EDEFNFDI_05016,BPDJECED_04485,DMLEFFEM_04762,EDHNABJO_03523,LBOLKHKO_00606
0,mills_cip_200__GCA_025783615,['a' 't' 'g' 't' 'c' 't' 'a' 't' 'a' 'c' 'a' '...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan ... nan nan nan],[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' 'a' 'g' 't' 'a' 'a' 'g' 'a' 't' '...,...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' ... 't' 'g' 'a'],['a' 't' 'g' ... 't' 'g' 'a'],[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' 't' 'c' 'c' 'c' 'a' 'g' 'a' 't' '...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...
1,kallonen_cip_200__ERR434265_pangenome_alignment,['a' 't' 'g' 't' 'c' 't' 'a' 't' 'a' 'c' 'a' '...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan ... nan nan nan],[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' 'a' 'g' 't' 'a' 'a' 'g' 'a' 't' '...,...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' ... 't' 'g' 'a'],['a' 't' 'g' ... 't' 'g' 'a'],[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' 't' 'c' 'c' 'c' 'a' 'g' 'a' 't' '...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...
2,kallonen_cip_200__ERR439575_pangenome_alignment,['a' 't' 'g' 't' 'c' 't' 'a' 't' 'a' 'c' 'a' '...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan ... nan nan nan],[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' 'a' 'g' 't' 'a' 'a' 'g' 'a' 't' '...,...,['-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' ... 't' 'g' 'a'],['a' 't' 'g' ... 't' 'g' 'a'],['-' '-' '-' '-' '-' '-' 'c' 't' 'g' 't' 'a' '...,['a' 't' 'g' 't' 'c' 'c' 'c' 'a' 'g' 'a' 't' '...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...
3,gladstone-cip-200__ERR4036619_pangenome_alignment,['a' 't' 'g' 't' 'c' 't' 'a' 't' 'a' 'c' 'a' '...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' ... 't' 'g' 'a'],[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' 'a' 'g' 't' 'a' 'a' 'g' 'a' 't' '...,...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' 't' 'c' 'c' 'c' 'a' 'g' 'a' 't' '...,['a' 't' 'g' 'c' 'g' 'g' 'c' 't' 't' 'g' 'c' '...,['a' 't' 'g' 'c' 'a' 'g' 't' 't' 't' 'g' 